In [16]:
#load "load.fsx"

open FSharpPlus
open FSharpPlus.Data
open Informedica.Utils.Lib.BCL

open Informedica.OpenAI.Lib

In [17]:
let test model =
    [
        for (text, exp) in Texts.testUnitTexts do
            let systemInput =
                let msg =
                    text
                    |> Prompts.System.systemDoseQuantityExpert
                    |> Message.system
                OpenAI.Chat.defaultChatInput model msg []

            let act, state =
                State.run (OpenAI.Extract.doseUnits model text) systemInput

            state |> OpenAI.Chat.print

            if act = exp then 1 else 0
            , (act, exp)
            , text
    ]

let calcScore = List.sumBy (fun (s, _, _) -> s)

let testResults =
    [
        OpenAI.Models.``gpt-3.5-turbo``
        //OpenAI.Models.``gpt-4-turbo-preview``
    ]
    |> List.map (fun model -> 
        printf $"- Testing: {model}: "
        let s = model |> test
        printfn $"score: {s |> calcScore}"
        model, s
    )

- Testing: gpt-3.5-turbo: 
## System:
You are an expert on medication prescribing, preparation and administration. You will give
exact answers. If there is no possible answer return an empty string.
You have to answer questions about a free text between ''' that describes the dosing of a medication.
You will be asked to extract structured information from the following text:

'''
alprazolam
6 jaar tot 18 jaar Startdosering: 0,125 mg/dag, éénmalig. Onderhoudsdosering: Op geleide van klinisch beeld verhogen met stappen van 0,125-0,25 mg/dosis tot max 0,05 mg/kg/dag in 3 doses. Max: 3 mg/dag. Advies inname/toediening: De dagdosis indien mogelijk verdelen over 3 doses.Bij plotselinge extreme slapeloosheid: alleen voor de nacht innemen; dosering op geleide van effect ophogen tot max 0,05 mg/kg, maar niet hoger dan 3 mg/dag.De effectiviteit bij de behandeling van acute angst is discutabel.
'''

ONLY respond if the response is present in the text. If the response cannot be extracted
respond w

In [11]:
testResults    
|> List.maxBy (fun (model, xs) -> xs |> calcScore)
|> fun (m, s) -> printfn $"\n\n## And the winner is: {m} with a high score: {s |> calcScore} from {Texts.testUnitTexts |> List.length}"

testResults
|> List.collect (fun (m, xs) -> xs)
|> List.groupBy (fun (_, _, t) -> t)
|> List.map (fun (txt, s) -> s |> calcScore, txt)
|> List.sortBy fst
|> List.iter (fun (s, t) ->
    printfn $"## Score: {s}"
    printfn $"## Text:\n{t}"
)



## And the winner is: gpt-3.5-turbo with a high score: 5 from 6
## Score: 0
## Text:

aprepitant
3 jaar tot 18 jaar en < 40 kg 2 maal per week 40 mg/dosis verdeeld over de week.

## Score: 2
## Text:

alprazolam
6 jaar tot 18 jaar Startdosering: 0,125 mg/dag, éénmalig. Onderhoudsdosering: Op geleide van klinisch beeld verhogen met stappen van 0,125-0,25 mg/dosis tot max 0,05 mg/kg/dag in 3 doses. Max: 3 mg/dag. Advies inname/toediening: De dagdosis indien mogelijk verdelen over 3 doses.Bij plotselinge extreme slapeloosheid: alleen voor de nacht innemen; dosering op geleide van effect ophogen tot max 0,05 mg/kg, maar niet hoger dan 3 mg/dag.De effectiviteit bij de behandeling van acute angst is discutabel.

## Score: 2
## Text:

acetylsalicylzuur
1 maand tot 18 jaar Startdosering:Acetylsalicylzuur: 30 - 50 mg/kg/dag in 3 - 4 doses. Max: 3.000 mg/dag.

## Score: 2
## Text:

paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een lee